In [3]:
import re
import time
import json
import yaml
import dicttoxml, xmltodict

from functools import partial
from pprint import pprint as pp

In [65]:
class timer():
    def __init__(self, message):
        self.message = message

    def __enter__(self):
        self.start = time.time()
        return None

    def __exit__(self, type, value, traceback):
        elapsed_time = (time.time() - self.start) * 1000
        print(self.message.format(elapsed_time))
        
class A():
    def __enter__(self):
        print('__enter__')
    
    def __exit__(self, type, value, traceback):
        print('__exit__')

with A():
    raise ValueError('WRONG!')

__enter__
__exit__


ValueError: WRONG!


## Magic attributes/methods

In [16]:
class A: # class A(object):
    """
    This is A
    """
    def __init__(self, name):
        self.name = name
    
    def foo(self):
        """
        foo
        """
        print('foo', self.name, self)

    
a = A('test')
b = A('test2')
# a.foo()
A.foo(a)
A.foo(b)

print(a)
print(b)
# print(dir(a))
# print(dir(A))
# print(dir(object))

foo test <__main__.A object at 0x7ff6717ba580>
foo test2 <__main__.A object at 0x7ff6717babe0>


In [19]:
print(a.__class__)
print(A.__class__, A.__bases__)
print(A.__doc__)
print(A.foo.__doc__)
help(A.foo)

<class '__main__.A'>
<class 'type'> (<class 'object'>,)

    This is A
    

        foo
        
Help on function foo in module __main__:

foo(self)
    foo



In [20]:
print(a.__str__())
print(str(a))

dir(a)
a.__dir__()

# xxx(a)
# a.__xxx__()

v = 42
print(v + 1)
print(v.__add__(1))

class Employee:
    def __init__(self, first_name=None, last_name=None, email=None):
        self._first_name = first_name
        self._last_name = last_name
        self._email = email
    
    def __bool__(self):
        return bool(self._first_name or \
               self._last_name or \
               self._email)
            
e = Employee()


if e:
    print('Not empty')
    # do some work
    
if e is not None: # None ~= null
    print('Is not None')
    # do some work

43
43
Is not None


#### operator overloading

In [7]:
class Q:
    def __init__(self, **params):
        self._params = params
    
    def __or__(self, other):
        self._params.update(other._params)
        return self

#     def __and__(self, other):
#         self._params.update(other._params)
#         return self
    
    def __str__(self):
        result = ''
        for k, v in  self._params.items():
            if result:
                result += ' OR '
#             if result:
#                 result += ' OR '
            result += f'{k}={repr(v)}'
        return result

filter = Q()
filter |= Q(first_name='John')
filter |= Q(last_name='Gonzalez')
filter |= Q(stuff=True)
filter |= Q(age=42)

print(filter)

first_name='John' OR last_name='Gonzalez' OR stuff=True OR age=42


#### `__dict__`

In [26]:
class A:
    attr1 = 'attr1' # static 
    
    def __init__(self, obj_attr1, obj_attr2):
        self.obj_attr1 = obj_attr1
        self.obj_attr2 = obj_attr2
        
a = A(1, 'abc')
a2 = A(2, 'xyz')

print(a.obj_attr1, id(a.obj_attr1))
print(a2.obj_attr1, id(a2.obj_attr1))
print(a.attr1, id(a.attr1))
print(a2.attr1, id(a2.attr1))

1 9756192
2 9756224
attr1 140696442113136
attr1 140696442113136


In [37]:
pp('Obj attrs: ')
pp(a.__dict__)
print()
pp('Class attrs: ')
pp(A.__dict__)

'Obj attrs: '
{'obj_attr1': 42, 'obj_attr2': 'abc', 'xxxx': 42}

'Class attrs: '
mappingproxy({'__dict__': <attribute '__dict__' of 'A' objects>,
              '__doc__': None,
              '__init__': <function A.__init__ at 0x7ff6780a1310>,
              '__module__': '__main__',
              '__weakref__': <attribute '__weakref__' of 'A' objects>,
              'attr1': 'XYZ'})


In [33]:
print(a.obj_attr1)
print(a.__dict__['obj_attr1'])
a.__dict__['obj_attr1'] = 42
print(a.obj_attr1)
A.attr1 = 'XYZ'
print(A.__dict__['attr1'])
print(a.attr1)

42
42
42
XYZ
XYZ


In [10]:
b = A(1, 'abc')
print(id(a.attr1), id(b.attr1))
print(id(a.obj_attr1), id(b.obj_attr1))
print(id(a.obj_attr2), id(b.obj_attr2))

140170420188720 140170420188720
9756192 9756192
140170524185776 140170524185776


In [11]:
a = A(1024, 'abc')
b = A(1024, 'abc'.upper().lower())
# c = A(1024, 'ab'+'c')
print(id(a.attr1), id(b.attr1))
print(id(a.obj_attr1), id(b.obj_attr1))
print(id(a.obj_attr2), id(b.obj_attr2))

140170420188720 140170420188720
140170420321552 140170420321296
140170524185776 140170420980848


In [12]:
b.attr1 = 42
print(id(a.attr1), id(b.attr1))

140170420188720 9757504


In [13]:
pp('Obj attrs: ')
pp(b.__dict__)
print()
pp('Class attrs: ')
pp(A.__dict__)

'Obj attrs: '
{'attr1': 42, 'obj_attr1': 1024, 'obj_attr2': 'abc'}

'Class attrs: '
mappingproxy({'__dict__': <attribute '__dict__' of 'A' objects>,
              '__doc__': None,
              '__init__': <function A.__init__ at 0x7f7bf819bdc0>,
              '__module__': '__main__',
              '__weakref__': <attribute '__weakref__' of 'A' objects>,
              'attr1': 'attr1'})


In [44]:
# Monkey patching
a.xxxx = 42
pp('Obj attrs: ')
pp(a.__dict__)
print(a.xxxx)


def super_lib():
    def super_func():
        print('foo')
    super_func = super_func
    
def profile(f):
    from time import time
    def deco(*args):
        start = time()
        f(*args)
        print('Elapsed: ', time()-start)
    return deco

print(id(super_lib.super_func))
super_lib.super_func = profile(super_lib.super_func)
print(id(super_lib.super_func))

'Obj attrs: '
{'obj_attr1': 42, 'obj_attr2': 'abc', 'xxxx': 42}
42
140696441656080


AttributeError: 'function' object has no attribute 'super_func'

In [15]:
a.foo = lambda x: print(x)

In [16]:
a.foo('bar')
pp('Obj attrs: ')
pp(a.__dict__)

bar
'Obj attrs: '
{'foo': <function <lambda> at 0x7f7bf8132550>,
 'obj_attr1': 1024,
 'obj_attr2': 'abc',
 'xxxx': 42}


#### get/set attributes

In [51]:
class A:
    def __getattr__(self, name):
        print('__getattr__')
#         if name in self.__dict__:
#             value = self.__dict__[name]
#         elif name in self.__class__.__dict__:
#             value = self.__class__.__dict__[name]
#         else:
#             value = super().__getattr__(name)                    
#         return value
        return 42
    
    def __setattr__(self, name, value):
        print('__setattr__')
        self.__dict__[name] = value + 1
        
    def __delattr__(self, name):
        if name in self.__dict__:
            del self.__dict__[name]
            
a = A()
a.boo = 42
print(a.boo)

__setattr__
43


In [56]:
def foo():
    print('foo')
foo()
foo.__call__()

class A:
    def __call__(self):
        print('__call__')
    
a = A()
a()


foo
foo
__call__


In [58]:
class lazy_object:
    '''
    Class for deferred instantiation of objects.  Init is called
    only when the first attribute is either get or set.
    '''

    def __init__(self, callable, *args, **kw):
        '''
        callable -- Class of objeсt to be instantiated or functionnn to be called
        *args -- arguments to be used when instantiating object
        **kw  -- keywords to be used when instantiating object
        '''
        self.__dict__['callable'] = callable
        self.__dict__['args'] = args
        self.__dict__['kw'] = kw
        self.__dict__['obj'] = None

    def initObj(self):
        '''
        Instantiate object if not already done
        '''
        if self.obj is None:
            self.__dict__['obj'] = self.callable(*self.args, **self.kw)

    def __getattr__(self, name):
        self.initObj()
        return getattr(self.obj, name)

    def __setattr__(self, name, value):
        self.initObj()
        setattr(self.obj, name, value)

    def __len__(self):
        self.initObj()
        return len(self.obj)

    def __getitem__(self, idx):
        self.initObj()
        return self.obj[idx]


In [59]:
class A:
    def __init__(self, num_elem):
        self.attr1 = list(range(num_elem))
        
a = lazy_object(A, num_elem=10**8)

print(a)

In [60]:
with timer('Elapsed: {}ms'):
   type(a.attr1)

with timer('Elapsed: {}ms'):
   type(a.attr1)

Elapsed: 1939.880609512329ms
Elapsed: 0.0064373016357421875ms


#### `__call__`

In [21]:
def foo():
    print('foo')

foo.__call__()

foo


In [22]:
class A:
    def __call__(self):
        print('called on object')

a = A()
a()

called on object


### Properties, hiding data

In [82]:
class A: # before
    
    def __init__(self, attr1=None, attr2=None):
        self.attr1 = attr1 # protected (client)
        self.attr2 = attr2 # protected (client)

class A:
    
    def __init__(self, attr1=None, attr2=None):
        self._attr1 = attr1 # protected (client)
        self._attr2 = attr2 # protected (client)
        self.__attr3 = attr2 # private (client&child)

    @property
    def attr1(self):
        print('get attr1')
        return self._attr1
        
    @attr1.setter
    def attr1(self, value):
        print('set attr1')
        if value > 0:
            self._attr1 = value
        else:
            raise ValueError('Invalid data')
       
    @property
    def attr2(self):
        print('get attr2')
        return self._attr2

a = A(attr2='abc')
a.attr1 = 42
print(a.attr1)
#a.attr1 = -1
print(a.attr2)
#a.attr2 = 'ABC'
#print(a.__attr3)

print(a.__dict__)
#print(a._A__attr3)
print(a._A__attr3)
a.__attr3 = 'ABC'
print(a.__attr3)
print(a.__dict__)


set attr1
get attr1
42
get attr2
abc
{'_attr1': 42, '_attr2': 'abc', '_A__attr3': 'abc'}
abc
ABC
{'_attr1': 42, '_attr2': 'abc', '_A__attr3': 'abc', '__attr3': 'ABC'}


In [68]:
a._attr2 = 'ABC'
print(a._attr2)

ABC




## Inheritance

In [97]:
class A:
    def __init__(self):
        self.attr1 = 'attrA'
        self.attrX = 'attrX'

    def foo(self):
        print('Hello from A.foo()')


class A1(A):
    def __init__(self):
        super().__init__()

class A2(A):
    
    def __init__(self):
        print('A2() called')
        
class B1(A1):
    pass

a1 = A1()
print(a1.attr1)

a2 = A2()
a2.foo()
print(a2.attr1)

attrA
A2() called
Hello from A.foo()


AttributeError: 'A2' object has no attribute 'attr1'

In [98]:
class A:

    def __init__(self):
        super().__init__()
    
    def foo(self):
        
        print('Hello from A.foo()')


class A1(A):
    def __init__(self):
        super().__init__()
        self.attrA1 = 'attrA1'
        
    def foo_a1(self):
        print('foo_a1() called')

        
class A2(A):
    def __init__(self):
        super().__init__()
        self.attrA2 = 'attrA2'

    def foo_a2(self):
        print('foo_a2() called')

        
a1 = A1()
a2 = A2()
print(a1.attrA1)
print(a2.attrA2)
a1.foo()
a2.foo()

a1.foo_a1()
a2.foo_a2()

attrA1
attrA2
Hello from A.foo()
Hello from A.foo()
foo_a1() called
foo_a2() called


In [99]:
  
class B(A1, A2):
    def __init__(self):
        supe r().__init__()
        self.attr1 = 'attrB'
        self.attr2 = 'attr2B'

b = B()
print(b.__dict__)


SyntaxError: invalid syntax (<ipython-input-99-2539cb78de9c>, line 3)

In [100]:
print(B.mro())

[<class '__main__.B'>, <class '__main__.A'>, <class 'object'>]


In [101]:
# Links:
# https://en.wikipedia.org/wiki/C3_linearization
# http://python-history.blogspot.com/2010/06/method-resolution-order.html

### Multiple Inheritance -> Mixins
#### A mix-in is a small class: 
- only defines a set of additional methods/attributes
- don't require their __init__ constructor to be called.


In [95]:
class JsonMixin(object):
    
    @classmethod
    def from_json(cls, data):
        obj = cls()
        attrs = json.loads(data)
        vars(obj).update(attrs)
        return obj
    
    def to_json(self):
        return json.dumps(vars(self))
    

class XMLMixin(object):
    
    @classmethod
    def from_xml(cls, data):
        obj = cls()
        attrs = xmltodict.parse(data)
        vars(obj).update(attrs)
        return obj
    
    def to_xml(self):
        return dicttoxml.dicttoxml(vars(self)).decode()
    

class YamlMixin(object):
    
    @classmethod
    def from_yaml(cls, data):
        kwargs = yaml.load(data)
        return cls(**kwargs)
    
    def to_yaml(self):
        return yaml.dump(vars(self))

In [102]:
class C(JsonMixin, XMLMixin, YamlMixin, B):
    pass
c = C()

print(c.to_json())
print(c.to_xml())
print(c.to_yaml())

print()
print(C.from_json(c.to_json()).to_json())
print(C.from_json(c.to_json()).to_json() == c.to_json())
# print(C.from_xml(c.to_xml()))

{}
<?xml version="1.0" encoding="UTF-8" ?><root></root>
{}


{}
True


#### Static methods

In [103]:
class A1:
    
    attr = 'attr_A'
    
    def bar(self):
        print('foo')
    
    @staticmethod
    def foo():
        print('foo: ', A1.attr)
        

    @classmethod
    def foo_class(cls):
        print('foo_class: ', cls.attr)

class B(A):
    attr = 'attr_B'
   
# A.bar()
A.foo()
A.foo_class()

B.foo()
B.foo_class()

TypeError: foo() missing 1 required positional argument: 'self'

### slots

In [116]:
class A:
    __slots__ = ['attr1', 'attr2']

a = A()
a.attr1 = 42
a.attr2 = '42'
print(a.attr1, a.attr2)

42 42


In [91]:
# a.attr3 = 'abc'

In [107]:
class A1:
    def __init__(self, attr1, attr2):
        self.attr1 = attr1
        self.attr2 = attr2

a1 = A1(42, '42')
print(a1.attr1, a1.attr2)

42 42


In [108]:
from sys import getsizeof
print(getsizeof(a), a.__slots__, getsizeof(a.__slots__))
print(getsizeof(a1), a1.__dict__, getsizeof(a1.__dict__))

48 ['attr1', 'attr2'] 72
48 {'attr1': 42, 'attr2': '42'} 104


In [113]:
with timer('Elapsed: {}ms'):
    for _ in range(10**6):
        a = A()
        a.attr1 = 42
        a.attr2 = '42'
        _ = a.attr1, a.attr2

with timer('Elapsed: {}ms'):
    for _ in range(10**6):
        a = A1(42, '42')
        _ = a.attr1, a.attr2


Elapsed: 306.05101585388184ms
Elapsed: 377.72488594055176ms


### Iterators

<img src="https://i.imgur.com/TyyXnx0.png">

### Descriptors

In [121]:
# Technically, descriptor is a class that supports the following methods: __set__[, __get__],__delete__

In [126]:
class Attribute:

    def __init__(self, initval=None, name='var'):
        self.val = initval
        self.name = name

    def __get__(self, obj, objtype):
        print('Retrieving', self.name)
        return self.val

    def __set__(self, obj, val):
        print('Updating', self.name)
        self.val = val
        
    def __delete__(self, obj):
        print('Deleting', self.name)
        self.val = None
        
class A:
    attr = Attribute(name='attr')
 
a = A()
a.attr = 42
print(a.attr)
del a.attr

# a1 = A()
# a.attr = 44
# a1.attr = 45
# print(a.attr)
# print(a1.attr)
# del a1.attr

Updating attr
Retrieving attr
42
Deleting attr
Updating attr
Updating attr
Retrieving attr
45
Retrieving attr
45
Deleting attr


In [202]:
class A:
    attr1 = (int, 0)
    attr2 = (str, '')

    def __getattribute__(self, name): # always called
        if name == '__dict__':
            return super().__getattribute__(name)
        obj_attrs = self.__dict__
        cls_attrs = vars(type(self))
        if name not in obj_attrs:
            if name in cls_attrs:
                _, default = cls_attrs[name]
                self.__dict__[name] = default
        return self.__dict__[name]

#     def __getattr__(self, name): # lookup fallback method
#         print('__getattr__', name)
#         obj_attrs = vars(self) # self.__dict__
#         cls_attrs = vars(type(self)) # self.__class__.__dict__
#         if name not in obj_attrs:
#             if name in cls_attrs:
#                 _, default = cls_attrs[name]
#                 self.__dict__[name] = default
#         return self.__dict__[name]
    
    def __setattr__(self, name, value):
        obj_attrs = vars(self)
        cls_attrs = vars(type(self))
        if name in cls_attrs:
            type_, default = cls_attrs[name]
            if isinstance(value, type_):
                self.__dict__[name] = value
            else:
                raise ValueError('Invalid type')

    def __delattr__(self, name):
        del self.__dict__[name]

In [206]:
a = A()
a1 = A()
print(a.attr1)
print(a.attr2)

a.attr1 = 32
a.attr2 = 'abc'

a1.attr1 = 33
a1.attr2 = 'xyz'

print(a.attr1, a1.attr1)
print(a.attr2, a1.attr2)

0

32 33
abc xyz


In [209]:
# a.attr1 = '42'
a.attr2 = 42

ValueError: Invalid type

In [221]:
class TypeCheckerMixin:

    def __setattr__(self, name, value):
        obj_attrs = vars(self)
        cls_attrs = vars(type(self))
        if name in cls_attrs:
            type_, default = cls_attrs[name]
            if isinstance(value, type_):
                self.__dict__[name] = value
            else:
                raise ValueError('Invalid type')

class A(TypeCheckerMixin):
    attr1 = (int, 0)
    attr2 = (str, '')
    attr3 = (bool, False)

a = A()

a.attr1 = 42
#a.attr1 = '42'

# But what if we want to check range for ints, regex amtch for strings, isclose() for floats?

In [282]:
class Descriptor:
    def __init__(self, name=None, default=None):
        self.name = name
        self.default = default

    def __set__(self, instance, value):
        instance.__dict__[self.name] = value

    def __get__(self, instance, objtype):
        if self.name not in instance.__dict__:
            instance.__dict__[self.name] = self.default
        return instance.__dict__[self.name]

    def __delete__(self, instance):
        raise AttributeError("Can't delete")

class Typed(Descriptor):
    type_ = object
    def __set__(self, instance, value):
        if not isinstance(value, self.type_):
            raise TypeError('Expected %s' % self.type_)
        super().__set__(instance, value)

# Specialized types
class Integer(Typed):
    type_ = int

class Float(Typed):
    type_ = float

class String(Typed):
    type_ = str


In [247]:
class A:
    attr1 = Integer(name='attr1')

a = A()
a.attr1 = 32
# a.attr1 = '32'

__set__ attr1 32


In [283]:
# Value checking
class Positive(Descriptor):
    def __set__(self, instance, value):
        if value < 0:
            raise ValueError('Expected >= 0')
        super().__set__(instance, value)

# More specialized types
class PosInteger(Integer, Positive):
    pass

class PosFloat(Float, Positive):
    pass

# Length checking
class Sized(Descriptor):
    def __init__(self, *args, maxlen, **kwargs):
        self.maxlen = maxlen
        super().__init__(*args, **kwargs)
        

    def __set__(self, instance, value):
        if len(value) > self.maxlen:
            raise ValueError('Too big')
        super().__set__(instance, value)

class SizedString(String, Sized):
    pass

# Pattern matching
class Regex(Descriptor):
    def __init__(self, *args, pattern, **kwargs):
        self.pattern = re.compile(pattern)
        super().__init__(*args, **kwargs)
    
    def __set__(self, instance, value):
        if not self.pattern.match(value):
            raise ValueError('Invalid string')
        super().__set__(instance, value)

class SizedRegexString(SizedString, Regex):
    pass


In [285]:
class A:
    attr1 = PosInteger(default=42)
    attr2 = PosFloat()
    attr3 = SizedRegexString(maxlen=11, pattern='\d{3}-\d{7}')

a = A()
print(a.attr1)
a.attr1 = 32
print(a.attr1)
a.attr2 = 0.1
a.attr3 = '067-9372129'

a1 = A()
a1.attr1 = 5
print(a1.attr1)


42
32
5


### Metaclasses


<img src="https://blog.ionelmc.ro/2015/02/09/understanding-python-metaclasses/instance-of.png">